### Generates Files For Database

The aim of this notebook is by using Obase Version 1's files, generating proper files for Obase Version 2 Database. 

Basically, the functions in this notebook generate 3 main files to the txtFiles directory:
    * CustomerMapping.txt
    * ItemMapping.txt
    * SalesTensor.txt
    
In order to generate these files, we need to get the following data files from Obase Version 1 and put them into the originalFiles directory. 
    * 81_7_24_7269_3392_Customers.txt
    * 81_7_24_7269_3392_Items.txt
    * 81_7_24_7269_3392_ItemsDs.txt
    * 81_7_24_7269_3392_Tensor.txt

#### CustomerMapping.txt
In the dataset provided by the Obase company, there is no order of customer id values. Values vary. But in our implementation, we need to represent customers by sequential values, from 0 to N (number of customers-1), which we named as CustomerIndex. 

This file contains the relation between customer indices and their real id values. First row of the file has field names (CustomerIndex and CustomerId) and the remaining rows contain customers' information, one at a time. The file has the following form:

    CustomerIndex   CustomerId
    0               516096
    1               1
    2               2793468
    3               3006459

CustomerIndex values will be used to determine each customer, so they should be unique and the values should start from 0 and go one by one.   

#### ItemMapping.txt
Similar to customers, we need to represent items with sequential numbers (namely ItemIndex). Also for some functions, we are planning to use the group information of the items, therefore we need the relation between items and the groups they belong to.

This file has the following form:

    ItemIndex	ItemId	ItemG3Index	ItemG3Id	ItemG3Ds	        ItemDs	
    0	        32823	 0	        470	        Tereyağlar	        SUTAS PASTORIZE 
    1	        32867	 1	        457	        Paketli Sosisler	ANTRIKOT SOSIS 400GR
    2	        32874	 2	        526	        Toz Tatlılar	    DR.OETKER 115GR
    24	        32970	-1	        -1	        Invalid Item	    BIBER KOY KG
    
In the provided dataset, some items do not have group information. We represent these items' ItemG3Index and ItemG3Id values as "-1" and IremG3Ds value as "Invalid Item". 
    
ItemIndex and ItemG3Index values will be used to determine items. ItemIndex values should be unique and the values should start from 0 and go one by one. ItemG3Index values should be unique to each item group and the values should start from 0 and go one by one (except the invalid items). 

#### SalesTensor.txt
This file contains the sales record of customers for a time period and has the following form:

    WeekIndex	DowIndex	HourIndex	ItemIndex	ItemG3Index	 CustomerIndex	Amount	
    0	        0	        13	        477	        -1	         404	        13.50
    0	        0	        13	        2847	    103	         404	        11.95
    0	        0	        13	        5250	    104	         404	        2.00
    42	        2	        6	        4474	    139	         3268	        1.75
    42	        2	        6	        2516	    31	         3268	        2.25
    42	        2	        6	        3277	    50	         3268	        9.95
    
In the above example, some sales of customers whose indices are 404 and 3268 are displayed. Each entry contains the time, item, customer, and price information. 

WeekIndex, DowIndex and HourIndex values are determined by setting a fixed starting date for the sales at Obase Version 1. By looking at the first entry of the above example, we can say customer with index 404 bought item with index 477 (which does not have group information) at the starting week, Monday at 13h00. 

### Database Structure
The database of Obase Version 2 will have the following structure (except weblog information of customers):

<img src="otherFiles/newDb.png">

In [ ]:
#%%writefile GenerateFilesForSql.py

import requests
import numpy as np
import scipy.io as sio
from scipy.sparse import *

def generateCustomerMapping(filename,outfilename,fieldnames):
    index = []
    data = []

    with open(filename) as inFile:
        for row, entry in enumerate(inFile, 0):
            index.append(int(row))
            data.append(int(entry))
    
    index = np.array(index)
    data = np.array(data)
            

    with open(outfilename, 'w') as outFile:
        #for i in range(len(fieldnames)):
        #    outFile.write('%s\t' % fieldnames[i])
        #outFile.write('\n')
        
        for i in range(len(index)):
            outFile.write('%d\t %d\n' % (index[i],data[i]))
            
            
def generateItemMapping(filename,filenameDs,outfilename,fieldnames):
    index = []
    itemId = []
    itemDs = []
    itemIdG3 = []
    itemDsG3 = []
    
    with open(filename) as inFile:
        for row, entry in enumerate(inFile, 0):
            index.append(int(row))
            itemId.append(int(entry))
    
    with open(filenameDs) as inFile:
        for row, entry in enumerate(inFile, 0):
            itemDs.append(entry)

    index = np.array(index)
    itemId = np.array(itemId)
    
    num = 0
    num2 = 0
    for i in range(len(itemId)):
        r = requests.get('http://212.57.2.68:93/api/database/urun?$select=IdUrunGrup3&$filter=IdUrun+eq+%d' % itemId[i])
        tempId = r.json()
        
        if len(tempId) == 0 or tempId[0]['IdUrunGrup3'] is None:
            tempId = -1
            tempDs = 'Invalid Item'
            num = num+1
        else:
            tempId = int(tempId[0]['IdUrunGrup3'])

            r = requests.get('http://212.57.2.68:93/api/database/urungrup3?$select=DsUrunGrup3&$filter=IdUrunGrup3+eq+%d' % tempId)
            tempDs = r.json()

            if len(tempDs) == 0 or tempDs[0]['DsUrunGrup3'] is None:
                tempDs = 'Invalid Item'
                num2 = num2 + 1
            else:
                tempDs = tempDs[0]['DsUrunGrup3']
                
        itemIdG3.append(int(tempId))
        itemDsG3.append(tempDs)
        
    print('Number of Invalid Items: %d %d' %(num,num2))
    
    indexG3 = []
    visited = []

    for i in range(len(index)):
        temp = itemIdG3[i]
        
        if int(temp) == -1:
            indexG3.append(-1)
        else:
            if temp not in visited:
                visited.append(temp)

            idx = visited.index(temp)
            indexG3.append(idx)

    with open(outfilename, 'w', encoding='utf-8') as outFile:
        #for i in range(len(fieldnames)):
        #    outFile.write('%s\t' % fieldnames[i])
        #outFile.write('\n')

        for i in range(len(index)):
            outFile.write('%d\t %d\t %d\t %d\t %s\t %s' % (index[i],itemId[i],indexG3[i],itemIdG3[i],itemDsG3[i],itemDs[i]))
            
            
def generateSalesTensor(filename,mappingfilename,outfilename,fieldnames):
    weekIndex = []
    dowIndex = []
    hourIndex = []
    itemIndex = []
    itemG3Index = []
    customerIndex = []
    amount = []

    itemMap = []
    with open(mappingfilename) as inFile2:
        for line2 in inFile2:
            values2 = line2.split('\t')
            itemMap.append(values2[2])    

    itemMap = itemMap[1:]

    with open(filename) as inFile:
        for line in inFile:
            values = line.split('\t')

            weekIndex.append(values[0])
            dowIndex.append(values[1])
            hourIndex.append(values[2])
            itemIndex.append(values[3])

            itemG3Index.append(itemMap[int(values[3])])

            customerIndex.append(values[4])
            amount.append(values[5])

    with open(outfilename, 'w', encoding='utf-8') as outFile:
            #for i in range(len(fieldnames)):
            #    outFile.write('%s\t' % fieldnames[i])
            #outFile.write('\n')

            for i in range(len(weekIndex)):
                outFile.write('%s\t %s\t %s\t %s\t %s\t %s\t %s' % (weekIndex[i],dowIndex[i],hourIndex[i],itemIndex[i],itemG3Index[i],customerIndex[i],amount[i]))

### Example Usages

In [ ]:
filename = 'originalFiles/81_7_24_7269_3392_Customers.txt'
outfilename = 'txtFiles/CustomerMapping.txt'
fieldnames = ['CustomerIndex', 'CustomerId']

generateCustomerMapping(filename,outfilename,fieldnames)

In [ ]:
filename = 'originalFiles/81_7_24_7269_3392_Items.txt'
filenameDs = 'originalFiles/81_7_24_7269_3392_ItemsDs.txt'
outfilename = 'txtFiles/ItemMapping.txt'
fieldnames = ['ItemIndex', 'ItemId', 'ItemG3Index', 'ItemG3Id', 'ItemG3Ds', 'ItemDs']

generateItemMapping(filename,filenameDs,outfilename,fieldnames)

In [ ]:
filename = 'originalFiles/81_7_24_7269_3392_Tensor.txt'
mappingfilename = 'txtFiles/ItemMapping.txt'
outfilename = 'txtFiles/SalesTensor.txt'
fieldnames = ['WeekIndex', 'DowIndex', 'HourIndex', 'ItemIndex', 'ItemG3Index', 'CustomerIndex', 'Amount']

generateSalesTensor(filename,mappingfilename,outfilename,fieldnames)